In [370]:
import pandas as pd

Загрузим табличку с нашими данными про длительность поездок такси! 

In [371]:
df = pd.read_csv('../data/taxi_dataset_with_target.csv', index_col=0)

df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km
id,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925


Попробуем построить модель Линейной Регрессии!

В качестве Выборки используйте таргетную колонку (*trip_duration*) и все признаки, кроме времени начала заказа (*pickup_datetime*)

Для начала воспользуемся "коробочным решением". Для этого создайте переменную model, положите в нее класс **LinearRegression** из модуля **linear_model** библиотеки **sklearn**.

Далее воспользуйтесь методом **fit**, положите в него через запятую: массив **X**, состоящий из признаков объектов (можно pandas dataframe'ом, а можно numpy-массивом), и массив **Y** с таргетами.

In [372]:
from sklearn.linear_model import LinearRegression

### Your code is here

X = df.drop(columns=['pickup_datetime', 'trip_duration'])
Y = df['trip_duration']

In [373]:
model = LinearRegression()
model.fit(X, Y)

LinearRegression()

Чтобы посмотреть на значения полученных коэффициентов модели, следует обратиться к атрибуту **coef_** класса линейной регрессии. Для просмотра свободного веса, к атрибуту **intercept_** 

In [374]:
### Your code is here
for column, coef in zip(X.columns, model.coef_):
    print(column, round(coef, 3))

print(round(model.intercept_, 3))

vendor_id 198.463
passenger_count 0.296
store_and_fwd_flag 56.469
distance_km 115.274
171.657


In [375]:
### Полученные коэффициенты будут являться ответом на задание 5 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

Теперь реализуйте функцию LinearRegressionByMatrix, которая будет принимать на вход 3 параметра:

Матрицу объект-признак **(X)**, вектор с ответами **(Y)**, булевый параметр **fit_intercept**, цель которого - добавить константный признак (состоящий из единичек), если True, и ничего не делать, если False.

Функция должна вернуть одномерный np.array объект с оцененными **$\beta_1, ..., \beta_n, \beta_0$**

Реализуйте решение через аналитическое решение в матрицах из лекции и практики:

$$
\beta^* = (X^T \cdot X)^{-1} \cdot X^T \cdot Y
$$

In [376]:
import numpy as np

### Your code is here

def LinearRegressionByMatrix(X: np.array, Y: np.array, fit_intercept: bool = True):
        """
        :param X: матрица объектов
        :param Y: вектор (матрица с 1 столбцом) ответов
        :param fit_intercept: добавить ли константную переменную в данные?
        
        :return: одномерный numpy-array с полученными коэффициентами beta
        """
        
        if fit_intercept:
                ones_column = np.ones((X.shape[0], 1))
                X_with_intercept = np.hstack((X, ones_column))
        else:
                X_with_intercept = X

        # Решение линейной регрессии с помощью матричных операций
        beta = np.dot(np.dot(np.linalg.inv(np.dot(X_with_intercept.T, X_with_intercept)), X_with_intercept.T), Y)
        
        return beta

In [377]:
X_train = df.drop(['trip_duration', 'pickup_datetime'], axis=1).values
Y_train = df['trip_duration'].values

LinearRegressionByMatrix(X_train, Y_train)

array([198.46313674,   0.29631295,  56.46912165, 115.27353763,
       171.65658145])

In [378]:
### Полученные коэффициенты будут являться ответом на задание 6 в ЛМС.
### Перечислите коэффициенты через запятую и пробел, округлив до 3 знаков после точки.
### Используйте точку в качестве десятичного разделителя.

coef = LinearRegressionByMatrix(X_train, Y_train)
for i, c in enumerate(coef):
    print(f"{i}: {round(c, 3)}")

0: 198.463
1: 0.296
2: 56.469
3: 115.274
4: 171.657


Получились ли коэффициенты такими же, как и в "коробочном" варианте?

Имея коэффициенты модели, можно восстановить предсказания для каждого объекта!

Сделайте это через операцию матричного произведения матрицы **X** и полученных коэффициентов (как результат работы *LinearRegressionByMatrix*)

In [379]:
### Your code is here

prediction = np.dot(np.hstack((X_train, np.arange(1, X_train.shape[0] + 1).reshape(-1, 1))), LinearRegressionByMatrix(X_train, Y_train))
prediction_mine = pd.DataFrame(prediction, index=Y.index, columns=['Y_mine'])
result = pd.concat([Y, prediction_mine], axis=1)
result

,trip_duration,Y_mine
id,,
id2875421,455.0,8.187747e+02
id2377394,663.0,8.273157e+02
id3858529,2124.0,1.725960e+03
id3504673,429.0,1.332209e+03
id2181028,435.0,1.469602e+03
...,...,...
id2376096,778.0,2.503858e+08
id1049543,655.0,2.503863e+08
id2304944,764.0,2.503869e+08
